In [ ]:
import os
import sys
import argparse
import numpy as np
import ruamel.yaml as yaml
import torch
import wandb
import logging
from logging import getLogger as get_logger
from tqdm import tqdm
from PIL import Image
import torch.nn.functional as F

from torchvision import datasets, transforms
from torchvision.utils import make_grid

from models import UNet, VAE, ClassEmbedder
from schedulers import DDPMScheduler, DDIMScheduler
from pipelines import DDPMPipeline
from utils import (
    seed_everything,
    init_distributed_device,
    is_primary,
    AverageMeter,
    str2bool,
    save_checkpoint,
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

In [ ]:
config = {
    "seed": 42,
    "device": device,
    
    "image_size": 128,
    "data_dir": "./data",
    "subset": 1.0,
    "batch_size": 4,
    "num_workers": 4,
    "output_dir": "experiments",
    "run_name": "default",

    'use_cfg': False,

    "unet_in_size": 128,
    "unet_in_ch": 3,
    "unet_ch": 128,
    "unet_num_res_blocks": 2,
    "unet_ch_mult": [1, 2, 2, 4],
    "unet_attn": [2, 3],
    "unet_dropout": 0.0,

    "num_train_timesteps": 100,
    "num_inference_steps": 100,
    "beta_start": 0.0002,
    "beta_end": 0.02,
    "beta_schedule": "linear",
    "variance_type": "fixed_small",
    "prediction_type": "epsilon",
    "clip_sample": True,
    "clip_sample_range": 1.0,

    "learning_rate": 1e-5,
    "weight_decay": 1e-4,
    "num_epochs": 10,

    "grad_clip": 1.0,
}

In [ ]:
seed_everything(config["seed"])

In [ ]:
device = torch.device(config["device"])

# Load Image

In [ ]:
transform = transforms.Compose(
        [
            transforms.Resize((config["image_size"], config["image_size"])),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
        ]
    )

In [ ]:
train_dataset = datasets.CIFAR10(
    config["data_dir"], train=True, transform=transform, download=True
)
# train_dataset = datasets.ImageFolder(config["data_dir"], transform=transform)
# sorted_indices = sorted(range(len(train_dataset)),
#                           key=lambda i: train_dataset.samples[i][0])
# subset_size = int(len(train_dataset) * config["subset"])
# subset_indices = sorted_indices[:subset_size]
# train_dataset = torch.utils.data.Subset(train_dataset, subset_indices)

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=config["batch_size"],
    shuffle=True,
    num_workers=config["num_workers"],
    pin_memory=True,
    sampler=None,
    drop_last=True,
)

In [ ]:
import matplotlib.pyplot as plt

images, labels = next(iter(train_loader))
print("Image shape:", images.shape)

grid = make_grid(images, nrow=4, normalize=True)

grid = (grid + 1.0) / 2.0
grid = grid.clamp(0,1)
grid = grid.permute(1, 2, 0).cpu().numpy()

plt.imshow(grid)
plt.show()

In [ ]:
total_batch_size = config["batch_size"] * 1

# Output Config

In [ ]:
if not os.path.exists(config["output_dir"]):
    os.makedirs(config["output_dir"])

if config["run_name"] is None:
    config["run_name"] = f"exp-{len(os.listdir(config['output_dir']))}"
else:
    config["run_name"] = f"exp-{len(os.listdir(config['output_dir']))}-{config['run_name']}"
output_dir = os.path.join(config["output_dir"], config["run_name"])
save_dir = os.path.join(output_dir, "checkpoints")
os.makedirs(output_dir, exist_ok=True)
os.makedirs(save_dir, exist_ok=True)

# Unet, DDPM Scheduler, Optimizer, LR Scheduler

### Unet

In [ ]:
unet = UNet(
    input_size=config["unet_in_size"],
    input_ch=config["unet_in_ch"],
    T=config["num_train_timesteps"],
    ch=config["unet_ch"],
    ch_mult=config["unet_ch_mult"],
    attn=config["unet_attn"],
    num_res_blocks=config["unet_num_res_blocks"],
    dropout=config["unet_dropout"],
    conditional=config["use_cfg"],
    c_dim=config["unet_ch"],
)

In [ ]:
num_params = sum(p.numel() for p in unet.parameters() if p.requires_grad)
print(f"Number of parameters: {num_params / 10 ** 6:.2f}M")

In [ ]:
unet = unet.to(device)

### Scheduler

In [ ]:
scheduler = DDPMScheduler(
    num_train_timesteps=config["num_train_timesteps"],
    num_inference_steps=config["num_inference_steps"],
    beta_start=config["beta_start"],
    beta_end=config["beta_end"],
    beta_schedule=config["beta_schedule"],
    variance_type=config["variance_type"],
    prediction_type=config["prediction_type"],
    clip_sample=config["clip_sample"],
    clip_sample_range=config["clip_sample_range"],
)

In [ ]:
scheduler = scheduler.to(device)

In [ ]:
scheduler_wo_ddp = scheduler.to(device)

### Optimizer, LR Scheduler

In [ ]:
optimizer = torch.optim.AdamW(
    unet.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"]
)
# TODO: setup scheduler
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=config["num_epochs"] * len(train_loader), eta_min=1e-6
)

### Train Steps

In [ ]:
num_update_steps_per_epoch = len(train_loader)
config["max_train_steps"] = config["num_epochs"] * num_update_steps_per_epoch

# Pipeline

In [ ]:
pipeline = DDPMPipeline(unet=unet, scheduler=scheduler)

# Experiment

### Set up

In [ ]:
print("***** Training arguments *****")
print("***** Running training *****")
print(f"  Num examples = {len(train_dataset)}")
print(f"  Num Epochs = {config['num_epochs']}")
print(f"  Instantaneous batch size per device = {config['batch_size']}")
print(
    f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}"
)
print(
    f"  Total optimization steps per epoch {num_update_steps_per_epoch}"
)
print(f"  Total optimization steps = {config['max_train_steps']}")

progress_bar = tqdm(range(config["max_train_steps"]))


### Start

In [ ]:
for epoch in range(config["num_epochs"]):
# for epoch in range(5):
    loss_m = AverageMeter()

    unet.train()
    scheduler.train()

    for step, (images, labels) in enumerate(train_loader):
        batch_size = images.size(0)

        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        noise = torch.randn_like(images)

        timesteps = torch.randint(
            0, scheduler.num_train_timesteps, (batch_size,), device=images.device
        )

        noisy_images = scheduler.add_noise(images, noise, timesteps)

        # # visualize image
        # grid = make_grid(images, nrow=4, normalize=True)
        # grid = (grid + 1.0) / 2.0
        # grid = grid.clamp(0,1)
        # grid = grid.permute(1, 2, 0).cpu().numpy()
        # plt.imshow(grid)
        # plt.title(f"Epoch {epoch}, Step {step}, Raw Image")
        # plt.show()

        # grid = make_grid(noisy_images, nrow=4, normalize=True)
        # grid = (grid + 1.0) / 2.0
        # grid = grid.clamp(0,1)
        # grid = grid.permute(1, 2, 0).cpu().numpy()
        # plt.imshow(grid)
        # plt.title(f"Epoch {epoch}, Step {step}, Noise Image")
        # plt.show()

        model_pred = unet(noisy_images, timesteps)

        if config["prediction_type"] == "epsilon":
            target = noise

        loss = F.mse_loss(model_pred, target)
        loss_m.update(loss.item())

        loss.backward()
        if config["grad_clip"]:
            torch.nn.utils.clip_grad_norm_(unet.parameters(), config["grad_clip"])
    
        param_before = next(unet.parameters()).clone()
        optimizer.step()
        param_after = next(unet.parameters()).clone()
        # print(f"Parameter change: {torch.sum(torch.abs(param_after - param_before)).item()}")

        lr_scheduler.step()

        progress_bar.update(1)

        if step % 100 == 0:
            print(
                f"Epoch {epoch+1}/{config['num_epochs']}, Step {step}/{num_update_steps_per_epoch}, Loss {loss.item()} ({loss_m.avg})"
            )
        # break
    
    unet.eval()
    generator = torch.Generator(device=device)
    generator.manual_seed(epoch + config["seed"])

    gen_images = pipeline(
        batch_size=config["batch_size"],
        num_inference_steps=config["num_inference_steps"],
        generator=generator,
        device=device,
    )

    # Convert PIL images to tensors
    gen_images_tensors = [transforms.ToTensor()(img) for img in gen_images]

    # Create a grid of images
    grid = make_grid(gen_images_tensors, nrow=4, normalize=True)

    # Convert the grid to a format suitable for display
    grid = (grid + 1.0) / 2.0
    grid = grid.clamp(0, 1)
    grid = grid.permute(1, 2, 0).cpu().numpy()

    # Display the grid
    plt.imshow(grid)
    plt.title(f"Epoch {epoch}, Generated Image")
    plt.show()